# Evolution of Einstein's equations

# Initial data , setting length of the domain and discretization


In [75]:
# Parameters
m = 1
A = 0.05#0.1124921875#0.125
run = 1

1

In [76]:
res=m;
N=2.0^m*1000.0#2.0^m*1000.0;#2.0^m*500.0;#N=2.0^m*500.0#2.0^m*100.0;
Xf=1.0;

dx=Xf/N;
dt=round(dx,digits=10);
Nt=2.0^m*1000.0#100.0*2^m*10
Tf=Nt*dt; #final time
print(Tf)

1.0

In [77]:
print(N)

2000.0

In [78]:
println("dx =", dx)
println("dt =", dt)
println("dx/dt =", dx/dt)

dx =0.0005
dt =0.0005
dx/dt =1.0


In [79]:
# Setting RESOLUTION

global dir = "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res$res"
#global dir = "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res"

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res1"

# Grid

In [80]:
ori=0.0;
initX1 = nothing
initX1=range(ori, stop=Xf, step=dx);

In [81]:
initX = range(round(ori-3.0*dx,digits=10), stop=Xf+3.0*dx, step=dx)

L=length(initX);
println("step size is  ", dx)

#T=range(dt,stop=Tf,step=dt)
#T=range(0,stop=Tf,step=dt)
println("the time step is ", dt)

step size is  0.0005
the time step is 0.0005


In [82]:
include("./ScalarField.jl");

# Initial data of m β Ψ and Ψ,x

In [83]:
using PyCall   
using Interpolations
using TickTock
using Dierckx

tick()
scipy = pyimport("scipy")
scipyinterpolate = pyimport("scipy.interpolate")


####

initm=zeros(L);
initbeta=zeros(L);
initpsi=zeros(L);
initderpsi=zeros(L);

state_array=[initm initbeta initpsi initderpsi initX];

#PSI
r0=0.7#0.3
sigma=0.3
initpsi[4:L-3] = init_gaussian(initX1,r0,sigma,A)

state_array[:,3] = initpsi
state_array = ghost(state_array)

#PSI,X FROM PSI
"""for i in 4:L-3
    initderpsi[i]=Der(state_array[:,:],i,3,initX)
end

state_array[:,4] = initderpsi
state_array = ghost(state_array)"""

#new
initderpsi[4:L-3] = init_gaussian_der(initX1,r0,sigma,A)
state_array[:,4] = initderpsi
state_array=ghost(state_array)

####


psi_func = Spline1D(initX[4:L-3], state_array[4:L-3,3],  k=4)
derpsi_func = Spline1D(initX[4:L-3], initderpsi[4:L-3],  k=4)#new

funcs = [psi_func, derpsi_func];

tock()

[ Info:  started timer at: 2023-04-06T10:07:56.522
[ Info:          0.354998162s: 354 milliseconds


In [84]:
#BETA
beta0=0
initbeta[4:L-3]=rk4wrapper(SFconstraint_beta,beta0,initX1,0,funcs)
state_array[:,2]=initbeta;
state_array=ghost(state_array);


#M
m0=0
initm[4:L-3]=rk4wrapper(SFconstraint_m,m0,initX1,0,funcs)
state_array[:,1]=initm;
state_array = ghost(state_array);

In [85]:
"""using Plots #showing initial data and the initial status of the horizon finder
plot(initX, state_array[:,1:4], xaxis="x",title="Initial Data", labels=["m" "beta" "psi" "psi,x"])"""

"using Plots #showing initial data and the initial status of the horizon finder\nplot(initX, state_array[:,1:4], xaxis=\"x\",title=\"Initial Data\", labels=[\"m\" \"beta\" \"psi\" \"psi,x\"])"

# Defining the initial state array for the evolution m β Ψ and Ψ,x

In [86]:
using CSV, Tables
#CSV.write(dir*"/time_step0.csv", Tables.table(transpose(Matrix(state_array))), writeheader=false)
CSV.write(dir*"/time_step0.csv", Tables.table(state_array), writeheader=false)

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res1/time_step0.csv"

In [87]:
timestep=0
criticality=0.0
explode=0.0
evol_stats = [run criticality A sigma r0 timestep explode]
monitor_ratio = zeros(L)
CSV.write("/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/parameters.csv", Tables.table(evol_stats), writeheader=true, header=["run", "criticality", "A", "sigma", "r0", "timestep", "explode"])

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/parameters.csv"

In [88]:
ginit=dt_scale(initX,state_array[:,1],state_array[:,2],dx)
println(update_dt(initX,state_array[:,1],state_array[:,2],dx,ginit)/dt)

1.0


 ## Time evolution

fixed grid

In [89]:
finaltime=0.05#length(T)-1
T = timeevolution(state_array,finaltime,dir,dt,run);

iteration 1 dt is 0.0005, time of iteration is 0.0005
iteration 2 dt is 0.0005, time of iteration is 0.001
iteration 3 dt is 0.0005, time of iteration is 0.0015
iteration 4 dt is 0.0005, time of iteration is 0.002
iteration 5 dt is 0.0005, time of iteration is 0.0025
iteration 6 dt is 0.0005, time of iteration is 0.003
iteration 7 dt is 0.0005, time of iteration is 0.0035
iteration 8 dt is 0.0005, time of iteration is 0.004
iteration 9 dt is 0.0005, time of iteration is 0.0045000000000000005
iteration 10 dt is 0.0005, time of iteration is 0.005000000000000001
iteration 11 dt is 0.0005, time of iteration is 0.005500000000000001
iteration 12 dt is 0.0005, time of iteration is 0.006000000000000002
iteration 13 dt is 0.0005, time of iteration is 0.006500000000000002
iteration 14 dt is 0.0005, time of iteration is 0.007000000000000003
iteration 15 dt is 0.0005, time of iteration is 0.007500000000000003
iteration 16 dt is 0.0005, time of iteration is 0.008000000000000004
iteration 17 dt is 0

 ## Movie of the evolution

In [633]:
using Plots, CSV, DataFrames
import GR
anim = @animate for n in 1:10:740#length(T)-1
    
    time_step=T[n+1]-T[n];
    IJulia.clear_output(true)
    
    functions3=DataFrame(CSV.File(dir*"/time_step$n.csv",header=false, delim=","))
    plot()
        plot1=functions3.Column1[4:L-3];
        plot2=functions3.Column2[4:L-3];
        plot3=functions3.Column3[4:L-3];
        plot4=functions3.Column4[4:L-3];
        plot5=functions3.Column5[4:L-3];
    d=round(T[n],digits=1)
    #plot!(X1, [plot1 plot2 plot3 plot4], labels=["m" "beta" "psi" "psi,x"],ylim=[-0.1,0.1], xlim=[0, 0.1]) |> display 
    plot!(initX[4:L-3], [plot1 plot2 plot3 plot4], labels=["m" "beta" "psi" "psi,x"],ylim=[-1,1])#, xlim=[0, 0.1]) |> display 
    
    end

gif(anim, "anim_res$res.gif", fps = 1)

LoadError: ArgumentError: "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res3/time_step1.csv" is not a valid file or doesn't exist

In [634]:
dir

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res3"